In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt

In [2]:
''' Load the scammer database tree for analysis '''
f = open("Scammer_Database/manifest")
scammer_database = json.load(f)
f.close()

In [3]:
''' Load the non scammer database tree for analysis '''
f = open("Non_Scammer_Database/manifest")
non_scammer_database = json.load(f)
f.close()

In [5]:
def list_to_dict(l,keyword):
    rval = {}
    for item in l:
        rval[item[keyword]] = item
    return rval

''' Load all the erc metadata '''
f = open('erc20_contract_address.json',encoding='utf-8')
erc20_contract_address = json.load(f)
erc20_contract_address['tokens'] = list_to_dict(erc20_contract_address['tokens'],'address')
f.close()

f = open('erc721_contract_address.json', encoding='utf-8')
erc721_contract_address = json.load(f)
erc721_contract_address['tokens'] = list_to_dict(erc721_contract_address['tokens'],'address')
f.close()

f = open('erc115_contract_address.json', encoding='utf-8')
erc115_contract_address = json.load(f)
erc115_contract_address['tokens'] = list_to_dict(erc115_contract_address['tokens'],'address')
f.close()

raw_contracts = {'erc20':erc20_contract_address,'erc115':erc115_contract_address,'erc721':erc721_contract_address}


In [6]:
def load_transactions(path,filename):
    f = open(path+filename)
    rval = json.load(f)
    f.close()
    return rval

In [185]:
from math import *
class Address:
    
    '''  
        We want to break up the data into sent and received 
        transactions and look for any sent-to-receive ratios 
        that could be meaningful.

        1)  Date and time of transactions 

        2) Frequency of transactions (send vs. received) 

        3) # of recurring transactions (# of transactions with the same address)

        4) Transaction category, external, internal, erc20, erc721 or erc1155

        5) Transaction asset 

        6) The ratio of average sent transaction to received transaction.

        7) The time between the received transaction and the sent 
        transaction (how quickly money gets moved to another wallet.  
        This feeds into recurrence as it will provide an understanding 
        of the money is getting sent to a consolidation wallet. We can 
        then isolate that wallet and see how many other addresses are 
        attached to the “consolidation”  wallet.)

        8) Length of wallet activity time between the first Block Number 
        transaction stamp and the last transaction stamp.  Let's consider 
        the wallet inactive after three months of inactivity. (it will be 
        interesting to understand how long a scam wallet stays active, and 
        if we identify a consolidation wallet, we will be able to identify 
        each time a scammer adds a new address to their portfolio and if 
        adding of new address corresponds with the beginning of current address 
        inactivity).  
    
    '''
    
    def __init__(self,**kwargs):
        self.path   = kwargs["path"]
        self.filename = kwargs["filename"]
        self.accumulate_transactions()
        self.compute_spreads()
        self.process_spreads()
        self.compute_mean_balance()
        
    def accumulate_transactions(self):
        ''' Accumulates and sorts the all the transactions '''
        self.history = {}
        addresses = load_transactions(self.path,self.filename)
        for address in addresses:
            ''' Incoming and Outgoing are separated calls to Alchemy's API '''
            out   = addresses[address]['Outgoing']
            inc   = addresses[address]['Incoming']
            ''' Accumulate and sort them by date (blockNum)  '''
            accum = []
            for trans in out:
                category = trans['category']
                if category == 'internal' or category == 'external':
                    accum.append([int(trans['blockNum'],16),-trans['value'],trans['to']])
                else:
                    pass
            for trans in inc:
                category = trans['category']
                if category == 'internal' or category == 'external':
                    accum.append([int(trans['blockNum'],16),trans['value'],trans['from']])
                else:
                    pass
            accum.sort(key=lambda t : t[0])
            self.history[address] = accum
            
    def compute_spreads(self):
        self.spreads = {}
        for address in self.history:
            addr_spread = {'Incoming':{},'Outgoing':{}}
            for transaction in self.history[address]:
                subdict = ''
                if transaction[1] > 0:
                    subdict = 'Incoming'
                else:
                    subdict = 'Outgoing'
                if(transaction[2] in addr_spread[subdict].keys()):
                    addr_spread[subdict][transaction[2]] += 1
                else:
                    addr_spread[subdict][transaction[2]]  = 1
            self.spreads[address] = addr_spread
    
    def process_spreads(self):
        
        def get_max_from_dict(dictionary):
            rval = 0
            for key in dictionary:
                if(rval<dictionary[key]):
                    rval = dictionary[key]
            return rval
        
        self.proc = {}
        for address in self.spreads:
            addr_dict = {}
            for typ in self.spreads[address]:
                addr_dict[typ] = {'Total':len(self.spreads[address][typ].keys()),
                                  'Max':get_max_from_dict(self.spreads[address][typ])
                                 }
            self.proc[address] = addr_dict
    
    def compute_mean_balance(self):
        self.means = {}
        ''' For each address we track the mean balance as well as the mean balance squared '''
        for address in self.history:
            self.means[address] = {}
            history = self.history[address]
            balance = 0
            mean_bal    = 0
            mean2_bal   = 0
            mean_tra    = 0
            mean2_tra   = 0
            mean_tbt    = 0
            mean2_tbt   = 0
            if len(history) == 0:
                self.means[address]['balances'] = {'std':0,'avg':0}
                self.means[address]['transactions'] = {'std':0,'avg':0}
                self.means[address]['time_between_transactions'] = {'avg':0,'std':0}
                continue
            birth_block   = history[0][0] 
            last_block    = history[-1][0]
            curr_block    = birth_block
            block_counter = 0
            for transaction in history:
                block_diff     = transaction[0] - curr_block + 1
                curr_block     = transaction[0]
                mean_bal      += balance*block_diff
                mean2_bal     += (balance*balance)*block_diff
                balance       += transaction[1]
                mean_tra      += transaction[1]
                mean2_tra     += transaction[1]*transaction[1]
                mean_tbt      += block_diff - 1
                mean2_tbt     += (block_diff - 1)*(block_diff - 1)
                block_counter += block_diff
            mean_bal  /= block_counter
            mean2_bal /= block_counter
            mean_tra  /= len(history)
            mean2_tra /= len(history)
            mean_tbt  /= len(history)
            mean2_tbt /= len(history)
            self.means[address]['balances']                  = {'avg':mean_bal,'std':sqrt(mean2_bal-mean_bal*mean_bal)}
            self.means[address]['transactions']              = {'avg':mean_tra,'std':sqrt(mean2_tra-mean_tra*mean_tra)}
            self.means[address]['time_between_transactions'] = {'avg':mean_tbt,'std':sqrt(mean2_tbt-mean_tbt*mean_tbt)}
        
    def write(self,**kwargs):
        path     = kwargs['path'] if 'path' in kwargs else ''
        filename = kwargs['filename']
        f = open(path+filename,"w")
        self.state = {'addresses': list(self.history.keys()),
                      'spreads':self.proc,
                      'means': self.means
                     }
        json.dump(self.state,f)
        f.close()
    
    def get_as_dicts(self):
        rval = []
        for address in self.history:
            rval.append({'address':address,
                         'spreads':self.proc[address],
                         'means':self.means[address]})
        return rval

In [158]:
Addr1 = Address(path="Scammer_Database/",filename=scammer_database['degenalgo.art'])

In [186]:
#Addr1.write(path='Processed_Scammer_Database/',filename='test_1')
path = 'Processed_Scammer_Database_2/'
processed_scammer_database = {}
i = 1
for scammer in scammer_database:
    print('Processing',scammer,"(",i,"/",len(scammer_database.keys()),")")
    try:
        Processor = Address(path='Scammer_Database/',filename=scammer_database[scammer])
        processed_scammer_database[scammer] = Processor.get_as_dicts()
    except:
        print("Scammer",scammer,"corrupted database")
    i +=1
sdb = open(path+'database','w')
json.dump(processed_scammer_database,sdb)
sdb.close()

Processing degenalgo.art ( 1 / 2569 )
Processing mingopack.org ( 2 / 2569 )
Processing artblocks-october.io ( 3 / 2569 )
Processing xxpp.org ( 4 / 2569 )
Processing rektguy.live ( 5 / 2569 )
Processing www.moodrollerrs.xyz ( 6 / 2569 )
Processing gangstamice.com ( 7 / 2569 )
Processing www.thewhitelistiio.xyz ( 8 / 2569 )
Processing bitcloutcaat.xyz ( 9 / 2569 )
Processing www.yu-gi-ym.xyz ( 10 / 2569 )
Processing yu-gi-ym.xyz ( 11 / 2569 )
Processing bevviesnft.xyz ( 12 / 2569 )
Processing vulcan-authing.xyz ( 13 / 2569 )
Processing kalebnft.live ( 14 / 2569 )
Processing alpacadabraaz.xyz ( 15 / 2569 )
Processing centralizedweb3exchange.pro ( 16 / 2569 )
Processing defi-chains.org ( 17 / 2569 )
Processing hypebearscluub.xyz ( 18 / 2569 )
Processing thegalagames.xyz ( 19 / 2569 )
Processing mint-seaside.com ( 20 / 2569 )
Processing www.fanggangnfft.xyz ( 21 / 2569 )
Processing champions.ink ( 22 / 2569 )
Processing projectmaybe.pre-minting.live ( 23 / 2569 )
Processing www.coolmonkees.

Processing hydrowhalescllub.xyz ( 209 / 2569 )
Processing www.eternal-vikinggs.xyz ( 210 / 2569 )
Processing www.angryapearmmy.xyz ( 211 / 2569 )
Processing pumma.xyz ( 212 / 2569 )
Processing www.asunasgiveawayys.xyz ( 213 / 2569 )
Processing www.punkscomiic.xyz ( 214 / 2569 )
Processing bloomddotxyz.xyz ( 215 / 2569 )
Processing thehaasbroothers.xyz ( 216 / 2569 )
Processing galaxy-egggs.xyz ( 217 / 2569 )
Processing www.thehummanoidsnft.xyz ( 218 / 2569 )
Processing themetakeey.xyz ( 219 / 2569 )
Processing www.stmachinenfft.xyz ( 220 / 2569 )
Processing sansound-nft.xyz ( 221 / 2569 )
Processing halonfts.xyz ( 222 / 2569 )
Processing berachaiin.xyz ( 223 / 2569 )
Processing oddstronauts.com ( 224 / 2569 )
Processing lordsocietynft.io ( 225 / 2569 )
Processing www.deepblluio.xyz ( 226 / 2569 )
Processing www.galaxisxyyz.xyz ( 227 / 2569 )
Processing www.castawayss-gg.xyz ( 228 / 2569 )
Processing www.nfteamms.xyz ( 229 / 2569 )
Processing www.dave-krugmaan.xyz ( 230 / 2569 )
Process

Processing memeland.mint-available.com ( 400 / 2569 )
Processing www.exosama.mint-available.com ( 401 / 2569 )
Processing www.degenheim.mint-available.com ( 402 / 2569 )
Processing www.atarixx.xyz ( 403 / 2569 )
Processing www.consortiumkeey.xyz ( 404 / 2569 )
Processing www.warpsound-aai.xyz ( 405 / 2569 )
Processing admit0010.xyz ( 406 / 2569 )
Processing nftuaae.xyz ( 407 / 2569 )
Processing wellyffriends.xyz ( 408 / 2569 )
Processing omgkirbydaao.xyz ( 409 / 2569 )
Processing www.thetronnwars.xyz ( 410 / 2569 )
Processing vince-vaan-dough.xyz ( 411 / 2569 )
Processing www.plurapecllub.xyz ( 412 / 2569 )
Processing steadysstacknft.xyz ( 413 / 2569 )
Processing superrrare.xyz ( 414 / 2569 )
Processing www.theposssessednft.xyz ( 415 / 2569 )
Processing mirrorpaass-xyz.xyz ( 416 / 2569 )
Processing www.big3ownershiip.xyz ( 417 / 2569 )
Processing digidaaigaku.xyz ( 418 / 2569 )
Processing metaverse-hhq.xyz ( 419 / 2569 )
Processing keunng.xyz ( 420 / 2569 )
Processing www.mftmkkuus.xyz

Processing www.auzki.com ( 586 / 2569 )
Processing auzki.com ( 587 / 2569 )
Processing www.mint-seaside.club ( 588 / 2569 )
Processing www.goblins.whitelist-spots.com ( 589 / 2569 )
Processing goblins.whitelist-spots.com ( 590 / 2569 )
Processing otherside.whitelist-spots.com ( 591 / 2569 )
Processing bapetaverse.whitelist-spots.com ( 592 / 2569 )
Processing www.bapetaverse.whitelist-spots.com ( 593 / 2569 )
Processing www.otherside.whitelist-spots.com ( 594 / 2569 )
Processing mint.kaijukingz.cyou ( 595 / 2569 )
Processing www.kaijukingz.cyou ( 596 / 2569 )
Processing gemxyz.biz ( 597 / 2569 )
Processing aradena.bond ( 598 / 2569 )
Processing www.mint.kaijukingz.cyou ( 599 / 2569 )
Processing www.aradena.bond ( 600 / 2569 )
Processing www.gemxyz.biz ( 601 / 2569 )
Processing www.everdome.cyou ( 602 / 2569 )
Processing www.ethereumtowers.bond ( 603 / 2569 )
Processing ethereumtowers.bond ( 604 / 2569 )
Processing www.astrobotsociety.cyou ( 605 / 2569 )
Processing akuworld.cyou ( 606 / 

Processing otherside2.xyz ( 791 / 2569 )
Processing www.hpos10i.com ( 792 / 2569 )
Processing deadfellaznfts.xyz ( 793 / 2569 )
Processing mint.jengas.io ( 794 / 2569 )
Processing www.invisiblefriends.gg ( 795 / 2569 )
Processing invisiblefriends.host ( 796 / 2569 )
Processing www.tritonpass.org ( 797 / 2569 )
Processing shrinemaiden.xyz ( 798 / 2569 )
Processing invlsiblefriends.io ( 799 / 2569 )
Processing www.societyofsatoshi.com ( 800 / 2569 )
Processing www.nudiecommunity.com ( 801 / 2569 )
Processing s7nstations.com ( 802 / 2569 )
Processing vandalcitycorps.com ( 803 / 2569 )
Processing exomon.shop ( 804 / 2569 )
Processing multibeasts-mints.netlify.app ( 805 / 2569 )
Processing pixelgenii.xyz ( 806 / 2569 )
Processing www.invlsiblefriends.io ( 807 / 2569 )
Processing digidaigaku.mintyournfts.live ( 808 / 2569 )
Processing 2-server-collab.land ( 809 / 2569 )
Processing connect-collabland.xyz ( 810 / 2569 )
Processing cryptolegends.store ( 811 / 2569 )
Processing crypto-usdt.top (

...

Processing www.corntonw.com ( 1719 / 2569 )
Processing parallelmint.com ( 1720 / 2569 )
Processing helixfreemint.com ( 1721 / 2569 )
Processing gangsterall.com ( 1722 / 2569 )
Processing www.helixfreemint.com ( 1723 / 2569 )
Processing 6529re.com ( 1724 / 2569 )
Processing www.eatcolective.com ( 1725 / 2569 )
Processing www.gangsterall.com ( 1726 / 2569 )
Processing sneakerheads-official.xyz ( 1727 / 2569 )
Processing munkonft.xyz ( 1728 / 2569 )
Processing bullsapesproject.xyz ( 1729 / 2569 )
Processing oxyaorigin-freemint.xyz ( 1730 / 2569 )
Processing genuineundeadnft.xyz ( 1731 / 2569 )
Processing vspass.xyz ( 1732 / 2569 )
Processing footiumnft.xyz ( 1733 / 2569 )
Processing 8pes-official.xyz ( 1734 / 2569 )
Processing neverfeartruth-freemint.xyz ( 1735 / 2569 )
Processing cloneforce-freemint.xyz ( 1736 / 2569 )
Processing theparallax-freemint.xyz ( 1737 / 2569 )
Processing phantom-network.xyz ( 1738 / 2569 )
Processing japanese-bapes.xyz ( 1739 / 2569 )
Processing www.helixmints.

Processing fancybearismetaverse.xyz ( 1899 / 2569 )
Processing krapopolisnft.live ( 1900 / 2569 )
Processing www.bbots19884.xyz ( 1901 / 2569 )
Processing www.theofficiala77x.xyz ( 1902 / 2569 )
Processing www.dudulabnfft.xyz ( 1903 / 2569 )
Processing thegodlynfft.xyz ( 1904 / 2569 )
Processing funkywolvees.xyz ( 1905 / 2569 )
Processing memelordnftts.xyz ( 1906 / 2569 )
Processing rplanetnfft.xyz ( 1907 / 2569 )
Processing www.crazykoalasnfft.xyz ( 1908 / 2569 )
Processing www.funkywolvees.xyz ( 1909 / 2569 )
Processing www.fuzzy-fighterrs.xyz ( 1910 / 2569 )
Processing www.cybertokyo-oox.xyz ( 1911 / 2569 )
Processing www.lostleadersnfft.xyz ( 1912 / 2569 )
Processing canisdaao.xyz ( 1913 / 2569 )
Processing moonshe-nfft.xyz ( 1914 / 2569 )
Processing www.rplanetnfft.xyz ( 1915 / 2569 )
Processing owangenfft.xyz ( 1916 / 2569 )
Processing weirdoghostganng.xyz ( 1917 / 2569 )
Processing www.woodlandrrz.xyz ( 1918 / 2569 )
Processing hexgo-nfft.xyz ( 1919 / 2569 )
Processing www.seaso

Processing www.cyber-kongz.xyz ( 2107 / 2569 )
Processing ethlizardsnft.live ( 2108 / 2569 )
Processing travala.live ( 2109 / 2569 )
Processing xquietth.mint300free.com ( 2110 / 2569 )
Processing homagames-mbmpost.premint.top ( 2111 / 2569 )
Processing www.penguinkarts.xyz ( 2112 / 2569 )
Processing www.fridaybeersnfft.xyz ( 2113 / 2569 )
Processing siphernft.live ( 2114 / 2569 )
Processing hiddenkittenctty.xyz ( 2115 / 2569 )
Processing pourkoko.xyz ( 2116 / 2569 )
Processing scrappypenguins-mint.com ( 2117 / 2569 )
Processing www.scrappypenguins-mint.com ( 2118 / 2569 )
Processing www.cryptoskulls-nft.xyz ( 2119 / 2569 )
Processing www.supducks-mint.xyz ( 2120 / 2569 )
Processing supducks-mint.xyz ( 2121 / 2569 )
Processing wolfgame-freemint.cyou ( 2122 / 2569 )
Processing www.wolfgame-freemint.cyou ( 2123 / 2569 )
Processing www.murakami-nft.cyou ( 2124 / 2569 )
Processing aptospre.sale ( 2125 / 2569 )
Processing aptossale.com ( 2126 / 2569 )
Processing 5ire.top ( 2127 / 2569 )
Proc

Processing bastardganpunks.xyz ( 2290 / 2569 )
Processing coinbasedefi.site ( 2291 / 2569 )
Processing tigerbob.freemint-fast.com ( 2292 / 2569 )
Processing airdrop-free.com ( 2293 / 2569 )
Processing airdrop-discord.com ( 2294 / 2569 )
Processing airdrop-discord.com.airdrop-collabland.cc ( 2295 / 2569 )
Processing ethdefi.run ( 2296 / 2569 )
Processing ethdefi.click ( 2297 / 2569 )
Processing cultivate.live ( 2298 / 2569 )
Processing silliesnft.com ( 2299 / 2569 )
Processing www.thesandboxgamme.xyz ( 2300 / 2569 )
Processing chibiheadnft.fast-premint.com ( 2301 / 2569 )
Processing www.ragstorichie.fast-premint.com ( 2302 / 2569 )
Processing www.chibiheadnft.fast-premint.com ( 2303 / 2569 )
Processing pixelvault.fast-premint.com ( 2304 / 2569 )
Processing www.pecland.fast-premint.com ( 2305 / 2569 )
Processing www.kitaroworld.fast-premint.com ( 2306 / 2569 )
Processing www.poobs.fast-premint.com ( 2307 / 2569 )
Processing www.otherside.fast-premint.com ( 2308 / 2569 )
Processing www.bl

Processing primekongplanet.mint-living.com ( 2466 / 2569 )
Processing rootroop.webs-mint.com ( 2467 / 2569 )
Processing runi.wl-spost.com ( 2468 / 2569 )
Processing boredapeyachtlclub.com ( 2469 / 2569 )
Processing nyolings.wl-spost.com ( 2470 / 2569 )
Processing aavev3serve.com ( 2471 / 2569 )
Processing liquidity-mining-token.com ( 2472 / 2569 )
Processing renga-whiteboxes.com ( 2473 / 2569 )
Processing aavev3top.com ( 2474 / 2569 )
Processing finiliar.web-minting.com ( 2475 / 2569 )
Processing acrocalypse.starts-mint.com ( 2476 / 2569 )
Processing platinumprimatez.com.silies.xyz ( 2477 / 2569 )
Processing www.smolverze.com.silies.xyz ( 2478 / 2569 )
Processing asiandisappointments.com.silies.xyz ( 2479 / 2569 )
Processing smolverze.com.silies.xyz ( 2480 / 2569 )
Processing outsadnft.web-minting.com ( 2481 / 2569 )
Processing metalegends.web-minting.com ( 2482 / 2569 )
Processing thelittles.starts-mint.com ( 2483 / 2569 )
Processing tron-mine.net ( 2484 / 2569 )
Processing tallylabs.

In [187]:
#Addr1.write(path='Processed_Scammer_Database/',filename='test_1')
path = 'Processed_Non_Scammer_Database_2/'
processed_non_scammer_database = {}
i = 1
for non_scammer in non_scammer_database:
    print('Processing',non_scammer,"(",i,"/",len(non_scammer_database.keys()),")")
    try:
        Processor = Address(path='Non_Scammer_Database/',filename=non_scammer_database[non_scammer])
        processed_non_scammer_database[non_scammer] = Processor.get_as_dicts()
    except:
        print("Non scammer",non_scammer,"corrupted database")
    i +=1
nsdb = open(path+'database','w')
json.dump(processed_non_scammer_database,nsdb)
nsdb.close()

Processing 0x00f8445d1a74f72d39154a6a75e475a135baabbb ( 1 / 1029 )
Processing 0x0124d0fa0dfb1430dfcff16ec6a96945e7a0bc10 ( 2 / 1029 )
Processing 0x013c0df3eea886b4a948ffd10459c641f13d865c ( 3 / 1029 )
Processing 0x03935ceee1db49524dd15faa37e8ccc92b1aaa71 ( 4 / 1029 )
Processing 0x05284f4fde5a8fbf284a05bf84d22933ceb10dfd ( 5 / 1029 )
Processing 0x0584c2125e35990e24affad82e20aebf6f67dc86 ( 6 / 1029 )
Processing 0x062ac5bb880629f90355d5c036656fa828dab22f ( 7 / 1029 )
Processing 0x0778eb4e4dc8946b7263b4a30a61d3e5bf748dca ( 8 / 1029 )
Processing 0x0805b0938a28de0f2df961f0666992ae22dbb67d ( 9 / 1029 )
Processing 0x0c1f218dbfb722467b019193edfcbd710aa455b8 ( 10 / 1029 )
Processing 0x0c24c489f9272136143e15f883ca04b46f4cb93b ( 11 / 1029 )
Processing 0x0d2579ce474351f66a311d9f7b66ff9ca989de58 ( 12 / 1029 )
Processing 0x0d7028abe96f6f8e782757fe2afb1b92475a82f3 ( 13 / 1029 )
Processing 0x0d8dff951c2aa73eb35fa6bee1dee20e8763b67d ( 14 / 1029 )
Processing 0x0d9df56b87ccff876e89eca59bc26a548ecbaa5f ( 1

Processing 0x4b67334b7d9ea083e032079b81495d132c7af508 ( 137 / 1029 )
Processing 0x4b8a339be06ff31752a806995b72e76e93dd9106 ( 138 / 1029 )
Processing 0x4dd2ef09e84465701872f51667a22543b75634b5 ( 139 / 1029 )
Processing 0x4e623c19ba39e1ce43af7ba43573755771966cd8 ( 140 / 1029 )
Processing 0x4eb8f1e82a05e891a70f3654daa8e4f2c99a07cc ( 141 / 1029 )
Processing 0x4f13d15cf8e028f6d62d7507a771100d3c00ade1 ( 142 / 1029 )
Processing 0x4f87e64e79840adf3f0b58fa9a87882dad2de00a ( 143 / 1029 )
Processing 0x4fd99198027c8809e0e8eeb760ca93656456db22 ( 144 / 1029 )
Processing 0x4fdc4ffcb2337abdd54281193dd43b43f683c9bf ( 145 / 1029 )
Processing 0x5008c29b7f65e1d8af00f58ad5b0ea17d359d617 ( 146 / 1029 )
Processing 0x5027d553c47ee1c824a38bf81cf91884a96e7a35 ( 147 / 1029 )
Processing 0x5051882d499ab55381bbcb1226ec7e5af762ebb2 ( 148 / 1029 )
Processing 0x505cf721fc5c7f0be5225b311a00ec72763e27ea ( 149 / 1029 )
Processing 0x5203528e041e7e22492498c3c13644a952863da3 ( 150 / 1029 )
Processing 0x52c1e10b5b3f23b9dc225

Processing 0x8f3ab4c0d4d41694fea84e30b1309b0628abdb99 ( 262 / 1029 )
Processing 0x9030824b858678135c2b1e9ba6ccca0d612e7ebc ( 263 / 1029 )
Processing 0x906c08a5bfe241e70b7b9b2ea03b8760afe50193 ( 264 / 1029 )
Processing 0x91aae0aafd9d2d730111b395c6871f248d7bd728 ( 265 / 1029 )
Processing 0x92111ae763f52d4d9fc0691f29f0bc27c7c24571 ( 266 / 1029 )
Processing 0x92d0c686f38138adf318298655e790fa15dba38b ( 267 / 1029 )
Processing 0x9336e547b1e1f77d2924c431d42b8e004c8cbb09 ( 268 / 1029 )
Processing 0x933b582082cff85d3003bf37236219d8048e3769 ( 269 / 1029 )
Processing 0x93bfd4e8e119e4cfa6fd285a3d6985854e46191b ( 270 / 1029 )
Processing 0x93e33bb973dffbf5de60e4fe5b790514861b655f ( 271 / 1029 )
Processing 0x94084fb3fa0f385d24fbee3255d2acdedc125fa0 ( 272 / 1029 )
Processing 0x945931d9e099ebdf5aa00fb49877aaf76bc24cc2 ( 273 / 1029 )
Processing 0x94b58e569c2ed08b2534ca41cd5782aa87c3983b ( 274 / 1029 )
Processing 0x94d66f5693a4032b3eb74d202e3ac2d4eb9ee747 ( 275 / 1029 )
Processing 0x95af81a7d06f76c6400e1

Processing 0xcee4ee149a1769da519725d9312ad635b738c29e ( 381 / 1029 )
Processing 0xcfc0f98f30742b6d880f90155d4ebb885e55ab33 ( 382 / 1029 )
Processing 0xcfd4a90f9ff60609c95d724abf4cf4a6a84367af ( 383 / 1029 )
Processing 0xd05fe132c016d5e427815142a897783f10bd1dc1 ( 384 / 1029 )
Processing 0xd0a2af86fdba370e8149234ef1d9b9966a579197 ( 385 / 1029 )
Processing 0xd0d1e00d089855fd7677283fa338be13fbc1aec4 ( 386 / 1029 )
Processing 0xd0ee542283f50b34fe4c25129f5f974c3d7899cb ( 387 / 1029 )
Processing 0xd13b46290f61db3e6638be2746e88f0c04ab883f ( 388 / 1029 )
Processing 0xd1801a085bbe164e7d0fa66902a4631dd46f4bdc ( 389 / 1029 )
Processing 0xd1ccf9d708dcdd60edebdda2eec2968b451c72b8 ( 390 / 1029 )
Processing 0xd201b1e5af0d2da0952bafe35cc4d531b12ca456 ( 391 / 1029 )
Processing 0xd3a211c74196644fe669e8b86bd994ead2a49499 ( 392 / 1029 )
Processing 0xd3bf91ad7342f0e2534371ed40c47a238e01378d ( 393 / 1029 )
Processing 0xd46ab1235757632cd8d6c0a9697ff028b59ec466 ( 394 / 1029 )
Processing 0xd48ecae8b906826b592c5

Processing 0x8d846712470befdff9a0c0e72eed039f710b18d2 ( 501 / 1029 )
Processing 0x7814f6529317df421b10d3c1dc104fe1dfe42a80 ( 502 / 1029 )
Processing 0x01ef78fb93c6294b21e2b06c88f6d1b17b07bdb7 ( 503 / 1029 )
Processing 0x68a58cfce6c9d1516b895c1bf58bd35d0d3a7361 ( 504 / 1029 )
Processing 0xd584df04c286ec1bbdf5fbb3dcbe6c2a6182d26e ( 505 / 1029 )
Processing 0xa600c9ffe32a292cd649ee9c8348af90df334c96 ( 506 / 1029 )
Processing 0x02c0678e744ce7f72e84e512ed7d11613713fc98 ( 507 / 1029 )
Processing 0x2b107f782fb36dcb7012e2eab3d136699159a249 ( 508 / 1029 )
Processing 0x6de9604324d062e83410ddba8767dc25c4e37b1c ( 509 / 1029 )
Processing 0xa626bd40a8c88f59a4cf9b1821a7bd71fad96285 ( 510 / 1029 )
Processing 0x7ceecec56b3668f06f11f82a17c85eafc8f8217e ( 511 / 1029 )
Processing 0x9b984d5a03980d8dc0a24506c968465424c81dbe ( 512 / 1029 )
Processing 0xbd6d0da8ec36dceefdb4fac802224cf7a871e535 ( 513 / 1029 )
Processing 0xcbc06ddb8a96d38843294cb730b84cdd1a6cdb22 ( 514 / 1029 )
Processing 0xecf124601650ab63fd289

Processing 0x8792dcf57d3a9b88bc9169d6d6a7a732c2e00ff0 ( 632 / 1029 )
Processing 0x43116917c16a6b9a5c71c62df1727d6184c649bc ( 633 / 1029 )
Processing 0x5e152a59dc314828179aca293c8e2800d8d4e7c1 ( 634 / 1029 )
Processing 0xe6dc5389881306341188ab5bfb93ba0d8580347d ( 635 / 1029 )
Processing 0x3b3d84207bb4947d78304e7eae3871c208feceb2 ( 636 / 1029 )
Processing 0x7a2503b55dfa46e9016cc68eeaaf946d01508e61 ( 637 / 1029 )
Processing 0xe5301ef65d09b639d8953df02bcec7b74c88c4eb ( 638 / 1029 )
Processing 0x9217290aa5425e127b8a40bb5d48401bb8e3e084 ( 639 / 1029 )
Processing 0x0205ef3b60b685933e3c9a8fc317f16c60091997 ( 640 / 1029 )
Processing 0xacbe74e26bd2fb9a56a8905e5a3fac9cb54647a6 ( 641 / 1029 )
Processing 0x0b38a2bc6951a235e6fb4ab9516c206bf51ab835 ( 642 / 1029 )
Processing 0x970957ce1f467ea1d45693119bf70dae30957978 ( 643 / 1029 )
Processing 0x79e77533e0732c3ac5a7c88ca72dad5f8a259e31 ( 644 / 1029 )
Processing 0xb52ede3199ae575bb7133ed3f10de311b01270e4 ( 645 / 1029 )
Processing 0xddbc9ead772fe25fa777b

Processing 0xa4bbe0bbb400029aa553d1a8ce7cd304bbbc014a ( 755 / 1029 )
Processing 0x8c6f9f4e5b7ae276bf58497bd7bf2a7d25245f64 ( 756 / 1029 )
Processing 0x727a538cbce9b2d5cf5d0bd1fde8dc5033428109 ( 757 / 1029 )
Processing 0x8819c442638140e09f5cc08afd19b172d083f159 ( 758 / 1029 )
Processing 0x56deaa6f9ef15addd03321640b132156163279cc ( 759 / 1029 )
Processing 0x2932aa54b63f816ada03b8c5c19bc25ac1060469 ( 760 / 1029 )
Processing 0x049443d2ec0df505ee2befd99df2fbea4824b461 ( 761 / 1029 )
Processing 0x159d829cb4e2d97c06ec2f977daa23acf8e45671 ( 762 / 1029 )
Processing 0x98e163be3d02a791a18cf0c0c508d136599d0af7 ( 763 / 1029 )
Processing 0x2e15d7aa0650de1009710fdd45c3468d75ae1392 ( 764 / 1029 )
Processing 0xfa1345c48f1590db98962f6d31ac7207253b8647 ( 765 / 1029 )
Processing 0x5b3d59d8c5d411ad1181e3ce64d4cf21e5e8c5cd ( 766 / 1029 )
Processing 0x7271fa1c1c8016e8906a682d546d93505151b4f3 ( 767 / 1029 )
Processing 0x17e46ccb764c6739af54d960d399df883fdbb888 ( 768 / 1029 )
Processing 0xafa4658aa1e59f438eb2d

Processing 0x0199ce3597ecab006e9f7ab92619540c48c307d1 ( 900 / 1029 )
Processing 0x9c67dd13645be82695e71c3ebb0e5767532609b5 ( 901 / 1029 )
Processing 0x7f455bc3d73c7464b0c315bf9bed09e1a2fa58d7 ( 902 / 1029 )
Processing 0x02011bad758f35239883f0a14af5c669ceebfa46 ( 903 / 1029 )
Processing 0x458dee1e99dc7929e4a153fba6946eeb0211dcfd ( 904 / 1029 )
Processing 0x530afa101bed324809d0738d45b38a669314ff54 ( 905 / 1029 )
Processing 0x0b93049323f681fa50f31e49d4e489c565f86f11 ( 906 / 1029 )
Processing 0x9cd2d27aa4b00b0e9195a9a38b2e1d55a33d4f55 ( 907 / 1029 )
Processing 0x6c76d2e8dc43b63d9952594514ed67eeec34218a ( 908 / 1029 )
Processing 0xed3c405156ab00a5e5b9ef46aaa282cf481f46bb ( 909 / 1029 )
Processing 0xb3aa50edb151147271a77df1ec8c67785cd9406e ( 910 / 1029 )
Processing 0xcd27ec5864044d6d9759ee52ae0f1064dcecb67e ( 911 / 1029 )
Processing 0xace57810549be61779f58a8b66d8725bf8aaff68 ( 912 / 1029 )
Processing 0x857aafd6efd502575dcce5103e942538094b29ab ( 913 / 1029 )
Processing 0xe901b80c49a1aa8d18fc5

In [71]:
print(raw_contracts['erc721']['tokens'])

{'0x0e3a2a1f2146d86a604adc220b4967a898d7fe07': {'chainId': 1, 'address': '0x0e3a2a1f2146d86a604adc220b4967a898d7fe07', 'name': 'Gods Unchained Cards', 'standard': 'erc721', 'symbol': 'CARD', 'logoURI': 'https://lh3.googleusercontent.com/yArciVdcDv3O2R-O8XCxx3YEYZdzpiCMdossjUgv0kpLIluUQ1bYN_dyEk5xcvBEOgeq0zNIoWOh7TL9DvUEv--OLQ=s60', 'extensions': {'link': 'https://godsunchained.com/?refcode=0x5b3256965e7C3cF26E11FCAf296DfC8807C01073', 'description': "Gods Unchained is a free-to-play, turn-based competitive trading card game in which cards can be bought and sold on the OpenSea marketplace. Players use their collection to build decks of cards, and select a God to play with at the start of each match. The goal of the game is to reduce your opponent's life to zero. Each deck contains exactly 30 cards. On OpenSea, cards can be sold for a fixed price, auctioned, or sold in bundles.", 'ogImage': 'https://godsunchained.com/assets/images/backgrounds/wide-hi-res-god.jpg'}}, '0x629cdec6acc980ebeeb